## Import libraries

In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import random
import numpy as np
import seaborn as sns

## Update parameter

Change this parameter (outlet number) to generate dataset for a different outlet

In [2]:
outlet_number = 3

## Import datasets

In [3]:
purchase_xls = pd.ExcelFile('Master Data (Purchase).xlsx')
sales_xls = pd.ExcelFile('Master Data (Sales).xlsx')

purchase_data_raw = pd.read_excel(purchase_xls, 'Purchase - Outlet {}'.format(outlet_number))
sales_data_raw = pd.read_excel(sales_xls, 'Sales - Outlet {}'.format(outlet_number))

mapping_df = pd.read_csv('Collated dataset - Sheet1.csv')

In [4]:
purchase_data_raw.head()

,SKU,Product Name - English,Product Name - Chinese,Date of Purchase,Quantity of Order Placed,Quantity of Order Received
0,A001,Enoki Mushroom,金针菇,2018/04/15,20.0,20.0
1,A001,Enoki Mushroom,金针菇,2018/04/16,15.0,15.0
2,A001,Enoki Mushroom,金针菇,2018/04/17,15.0,15.0
3,A001,Enoki Mushroom,金针菇,2018/04/18,20.0,20.0
4,A001,Enoki Mushroom,金针菇,2018/04/19,20.0,20.0


In [5]:
sales_data_raw.head()

,SKU,Product - English Name,Product - Chinese Name,Date of Sale,Amount Sold
0,A001-1,Enoki Mushroom,金针菇,2018/01/01,26
1,A001-1,Enoki Mushroom,金针菇,2018/01/02,4
2,A001-1,Enoki Mushroom,金针菇,2018/01/03,3
3,A001-1,Enoki Mushroom,金针菇,2018/01/04,3
4,A001-1,Enoki Mushroom,金针菇,2018/01/05,12


In [6]:
mapping_df.head()

,product_sku,sales_spec,raw_material_sku,raw_material_name
0,A001-1,2.0,A001,Enoki mushroom
1,A001-2,1.0,A001,Enoki mushroom
2,A013,1.0,A001,Enoki mushroom
3,A002-1,1.0,A002,Oyster mushroom
4,A002-2,0.5,A002,Oyster mushroom


In [7]:
product_sku_list = sales_data_raw['SKU'].unique()
product_sku_list

array(['A001-1', 'A002-1', 'A003-1', 'A004-1', 'A005-1', 'A006-1', 'A013',
       'A014', 'A011-1', 'A007-1', 'A012-1', 'A008-1', 'A009-1', 'A010-1',
       'A002-2', 'A004-2', 'A006-2', 'A001-2', 'A003-2', 'A005-2'],
      dtype=object)

In [8]:
raw_materials_sku_list = list(set(purchase_data_raw['SKU']))
raw_material_sales = {}
for raw_materials_sku in raw_materials_sku_list:
    raw_material_sales[raw_materials_sku] = {}

pd.DataFrame.from_dict(raw_material_sales)

,A007,A012,A002,A001,A004,A006,A008,A010,A011,A009,A003,A005


In [9]:
print(raw_materials_sku_list)

['A007', 'A012', 'A002', 'A001', 'A004', 'A006', 'A008', 'A010', 'A011', 'A009', 'A003', 'A005']


In [10]:
for index, row in sales_data_raw.iterrows():
#     print(row)
    sku = row['SKU']
    eng_name = row['Product - English Name']
    date_of_sale = row['Date of Sale']
    amount_sold = row['Amount Sold']
        
    if sku in raw_materials_sku_list:
        if date_of_sale not in raw_material_sales[sku]:
            raw_material_sales[sku][date_of_sale] = 0
        raw_material_sales[sku][date_of_sale] += amount_sold
        continue
    
    # find raw materials
    raw_materials = mapping_df[mapping_df['product_sku'] == sku]
    for raw_material_i, raw_material_row in raw_materials.iterrows():
        raw_material_sku = raw_material_row['raw_material_sku']
        raw_material_amt = raw_material_row['sales_spec']
        if date_of_sale not in raw_material_sales[raw_material_sku]:
            raw_material_sales[raw_material_sku][date_of_sale] = 0
        raw_material_sales[raw_material_sku][date_of_sale] += raw_material_amt * amount_sold

raw_material_sales_df = pd.DataFrame.from_dict(raw_material_sales).reset_index().rename(columns={"index": "date"})
raw_material_sales_melted_df = pd.melt(raw_material_sales_df, id_vars=['date'], value_vars=raw_materials_sku_list).rename(columns={"variable": "raw_material_sku"})
raw_material_sales_final_df = raw_material_sales_melted_df.fillna(0)

raw_material_sales_final_df

,date,raw_material_sku,value
0,2018/01/01,A007,1.765
1,2018/01/02,A007,0.190
2,2018/01/04,A007,0.315
3,2018/01/05,A007,0.695
4,2018/01/06,A007,1.265
...,...,...,...
12163,2020/08/25,A005,0.000
12164,2020/03/09,A005,0.000
12165,2020/11/17,A005,0.000
12166,2020/04/02,A005,0.000


In [11]:
raw_material_sales_final_df['date']= pd.to_datetime(raw_material_sales_final_df['date'])
raw_material_sales_final_df.dtypes

date                datetime64[ns]
raw_material_sku            object
value                      float64
dtype: object

In [12]:
for raw_material_sku in raw_materials_sku_list:
    requirements_for_each_raw_material = raw_material_sales_final_df[raw_material_sales_final_df['raw_material_sku'] == raw_material_sku][['date', 'value']].rename(columns={"date": "ds", "value": "y"})
    requirements_for_each_raw_material.to_csv('churned datasets/outlet{}_{}.csv'.format(outlet_number, raw_material_sku), index = False)